In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

#### yellow taxi: feature selection

#os.chdir("/Users/shuozhang/Desktop/capstone/yellow taxi")
yellow_raw=pd.read_csv("yellow_tripdata_2014-05.csv", index_col=False)
yellow_raw.columns = [c.replace(" ", "") for c in yellow_raw.columns] # remove space in column name
yellow=yellow_raw[['pickup_datetime', 'pickup_latitude', 'pickup_longitude']]
yellow.columns=['Date', 'Lat', 'Lon'] # rename for future merge
yellow["Type"]="yellow" # EDA analysis label
yellow["Date"]=yellow["Date"].str[0:13].replace("\s+","-", regex=True) 


/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


IndexError: list index out of range

In [6]:
#### green taxi: feature selection

#os.chdir("/Users/shuozhang/Desktop/capstone/green taxi")
green_raw=pd.read_csv("green_tripdata_2014-05.csv")
green_raw.columns=[c.replace(" ", "") for c in green_raw.columns]  
green_raw['lpep_pickup_datetime']=green_raw['VendorID'] # the first 3 columns are mislabeled
green_raw.index=np.arange(green_raw.shape[0])  # index is wrong
green=green_raw[['lpep_pickup_datetime', 'Pickup_longitude', 'Pickup_latitude']]
green.columns=['Date', 'Lon', 'Lat']
green["Type"]="green"
green["Date"]=green["Date"].str[0:13].replace("\s+","-", regex=True)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [8]:
#### uber: feature selection
uber=pd.read_csv("uber-raw-data-may14.csv")
uber.columns=['Date', 'Lat', 'Lon', 'Base']
uber.drop(['Base'], axis=1, inplace=True)
uber["Type"]="uber"
uber["Date"]=pd.to_datetime(uber["Date"]).dt.strftime('%Y-%m-%d-%H')

In [10]:
data=pd.concat([yellow, green, uber], axis=0)

In [44]:
weather=pd.read_csv("weather_2014_05.csv", index_col=False)
weather.columns=[c.replace(" ", "") for c in weather.columns]
weather["YR--MODAHRMN"]=pd.to_datetime(weather["YR--MODAHRMN"],format="%Y%m%d%H%M")
weather["YR--MODAHRMN"]=pd.to_datetime(weather["YR--MODAHRMN"]).dt.strftime('%Y-%m-%d-%H')
weather = weather[['YR--MODAHRMN','TEMP','PCP01']]
weather.columns=['Date', 'Temp', 'Rain']
weather['Rain'] = weather['Rain'].replace('*****',"NA")
weather['Temp'] = weather['Temp'].replace('****',"NA")


In [52]:
dataall=pd.merge(data, weather, how='left', on=['Date'])
dataall.columns

Index(['Date', 'Lat', 'Lon', 'Type', 'Temp', 'Rain'], dtype='object')

In [139]:
data = dataall.loc[(dataall['Lat']>= 40) & (dataall['Lat']<= 41.5) & (dataall['Lon']>= -75) & (dataall['Lon'] <= -72)]
data.shape # still too many rows 23080677

(23080677, 6)

In [172]:
#dimension reduction to sample rows from the data by weights
yellow_s = yellow.sample(n = 3000,replace= False)
green_s = green.sample(n= 1000,replace= False)
uber_s = uber.sample(n = 1000, replace = False)

In [173]:
data_s  = pd.concat([yellow_s, green_s, uber_s], axis=0)
dataall_s = pd.merge(data_s, weather, how='left', on=['Date'])
data = dataall_s.loc[(dataall_s['Lat']>= 40) & (dataall_s['Lat']<= 41.5) & \
                     (dataall_s['Lon']>= -75) & (dataall_s['Lon'] <= -72)]
data.shape #7003 rows

(7003, 6)

In [174]:
data = data.loc[(data['Temp'] != 'NA') & (data['Rain'] != 'NA')]

In [243]:
data.shape #5665

(5665, 6)

In [176]:
df = data[['Lat','Lon']]

In [208]:
#### weather: feature selection
#### use google map api to read zipcode
from geopy.geocoders import Nominatim
import re
import time
import csv
geolocator = Nominatim()
zipcode = []
#zipcode_all = []

for n in range(2431,df.shape[0]):
    loc = ','.join(map(str, df.iloc[n]))
    location = geolocator.reverse(loc, timeout=50)
    if location.address == None:
        zipcode ="NaN"
    else:
        num = re.findall(r"\D(\d{5})\D", location.address)
        if len(num) == 0:
             zipcode = "NaN"
        else:
             zipcode = num[0]
    output = str(loc) + "," + zipcode
    time.sleep(1)
    with open(r"zipcode_out3.csv", 'a') as f:
        writer = csv.writer(f,delimiter='\t')
        writer.writerow([output])

In [219]:
zipcode1=pd.read_csv('zipcode_out.csv', sep=',', index_col=False, dtype={'zipcode':'S10'},header=None)
zipcode2=pd.read_csv('zipcode_out2.csv', sep=',', index_col=False, dtype={'zipcode':'S10'},header=None)
zipcode3=pd.read_csv('zipcode_out3.csv', sep=',', index_col=False, dtype={'zipcode':'S10'},header=None)


In [245]:
result = pd.concat([zipcode1,zipcode2,zipcode3])
result.columns = ["Lat","Lon","zipcode"]

In [246]:
result_1 = pd.to_numeric(result["Lat"], errors='coerce')
result_2 = pd.to_numeric(result["Lon"], errors='coerce')
result_3 = pd.to_numeric(result["zipcode"], errors='coerce')

In [247]:
df_2 = pd.concat([result_1, result_2], axis=1)
zipcode = pd.concat([df_2, result_3], axis=1)
data = pd.DataFrame(data)

In [248]:
may = pd.merge(data, zipcode, how='left', on=['Lat','Lon'])

In [249]:
may.shape

(6918, 7)

In [252]:
nyc_zipcode=pd.read_csv('nyc_zipcode.csv', sep='\t', index_col=False, dtype={'Zipcode':'S10'})
may=may[may['zipcode'].isin(nyc_zipcode['zipcode'])] # filter nyc-only zipcode
may_aggre=may.groupby(['Date', 'Temp', 'Rain', 'zipcode']).size().reset_index(name='count')
#may_aggre.to_csv('may_final_data.csv', sep='\t', index_col=False)

In [253]:

#data_final=pd.read_csv('may_final_data.csv', sep='\t', encoding='utf-8', index_col=0, dtype={'zipcode':'S10'})
data_final = may_aggre
date=pd.to_datetime(data_final['Date'], format='%Y-%m-%d-%H')
#data_final['Month']=date.dt.month
data_final['Day']=date.dt.day
data_final['Hour']=date.dt.hour
temp=pd.DatetimeIndex(date)
data_final['Weekday']=temp.weekday
data_model=data_final[['Weekday', 'Hour', 'Temp', 'Rain', 'zipcode', 'count']]
#data_model.to_csv('nycmodeldata.csv', sep='\t', encoding='utf-8', index_col=False)

In [258]:
data_model['rush'] = data_model.apply(lambda row: (row['Hour']>=7 )&(row['Hour']<= 10),axis = 1)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [262]:
data_model.shape #(2343, 7)

(2343, 7)

In [264]:
data_model.to_csv('nycmodeldata.csv', sep='\t', encoding='utf-8')

In [281]:
a = pd.merge(data_final,may, how ='left', on =['zipcode','Date','Temp','Rain'])

In [283]:
a.to_csv('maydata.csv', sep='\t', encoding='utf-8')